In [217]:
import pandas as pd
import pyarrow as pa
import json

In [229]:
data = pd.read_csv('./flights-10k.csv', encoding='utf-8', dtype={'FL_DATE': 'str', 'ARR_TIME': 'str', 'DEP_TIME': 'str'})

In [232]:
arr_time = data.FL_DATE + data.ARR_TIME.replace('2400', '0000')
data['ARRIVAL'] = pd.to_datetime(arr_time, format='%Y%m%d%H%M')

dep_time = data.FL_DATE + data.DEP_TIME.replace('2400', '0000')
data['DEPARTURE'] = pd.to_datetime(dep_time, format='%Y%m%d%H%M')

In [238]:
df = data.drop(['FL_DATE', 'ORIGIN', 'DEST', 'ARR_TIME', 'DEP_TIME'], axis=1).dropna()

In [239]:
df.head()

,DEP_DELAY,ARR_DELAY,AIR_TIME,DISTANCE,ARRIVAL,DEPARTURE
0,72.0,43.0,109.0,954,2016-12-01 23:59:00,2016-12-01 21:39:00
1,-4.0,-9.0,201.0,1437,2016-12-01 12:33:00,2016-12-01 09:51:00
2,90.0,92.0,191.0,1258,2016-12-01 16:12:00,2016-12-01 12:32:00
3,-4.0,-31.0,147.0,1258,2016-12-01 09:38:00,2016-12-01 06:56:00
4,-4.0,3.0,137.0,1012,2016-12-01 12:35:00,2016-12-01 08:51:00


In [243]:
from datetime import date, datetime

def json_serial(obj):
    return int(obj.strftime("%s"))

d = {}
for column in df.columns:
    d[column]=list(df[column])

with open('flights-10k.json', 'w') as f:
    json.dump(d, f, default=json_serial, separators=(',', ':'))

In [166]:
table = pa.Table.from_pandas(df)

In [167]:
writer = pa.RecordBatchFileWriter('flights-10k.arrow', table.schema)
writer.write(table)
writer.close()

In [168]:
!ls -lah

total 377632
drwxr-xr-x@ 13 domoritz  staff   416B Jun  8 14:38 .
drwxr-xr-x  24 domoritz  staff   768B Jun  8 01:20 ..
-rw-r--r--@  1 domoritz  staff   6.0K Nov 29  2017 .DS_Store
drwxr-xr-x   3 domoritz  staff    96B Jun  7 23:57 .ipynb_checkpoints
-rw-r--r--@  1 domoritz  staff    28M Nov 28  2017 564230852_T_ONTIME.csv
-rw-r--r--@  1 domoritz  staff   6.3M Nov 28  2017 564230852_T_ONTIME.zip
-rw-r--r--   1 domoritz  staff   7.0K Jun  8 14:38 convert.ipynb
-rw-r--r--   1 domoritz  staff   557K Jun  8 14:39 flights-10k.arrow
-rw-r--r--   1 domoritz  staff   400K Nov 29  2017 flights-10k.csv
-rw-r--r--   1 domoritz  staff    31M Nov 29  2017 flights-1m.csv
-rw-r--r--   1 domoritz  staff   7.7M Feb 13 21:44 flights-200k.csv
-rw-r--r--   1 domoritz  staff    92M Nov 29  2017 flights-3m.csv
-rw-r--r--@  1 domoritz  staff    18M Feb 13 21:44 flights-500k.csv
